les imports :

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.7 MB/s eta 0:00:00


In [4]:
!pip install pysam

  Obtaining dependency information for pysam from https://files.pythonhosted.org/packages/3e/36/0d478e81f4687e87be1b1cdb3451d41c088669687f28c03ac60ffefb2f71/pysam-0.22.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.1/25.1 MB 8.1 MB/s eta 0:00:00m eta 0:00:010:00:01


In [6]:
import numpy as np
from Bio import SeqIO
from collections import Counter
#import pysam

In [7]:
def read_fasta_file(fasta_path):
    fasta_seq = []
    reads = SeqIO.parse(fasta_path, "fasta")
    for read in reads:
        read_seq = str(read.seq)
        fasta_seq.append(read_seq)
    return fasta_seq

In [8]:
fasta_path = "GCF_000002765.5_GCA_000002765_genomic.fna"
genome = read_fasta_file(fasta_path)[0]

In [9]:
len(genome)

640851

In [10]:
reads = SeqIO.parse("single_Pfal_dat.fq", "fastq")

In [11]:
Alphabet=['A','C','T','G']

In [12]:
#### J'ai fait les imports sur mon pc car sinon ca marche pas (Lilian)

In [13]:
#########Constitution des k-mer : il faut traiter les reads les uns après les autres en les écrasant successivement pour ne pas faire crash les pc en surchargeant la mémoire
######### En effet les reads sont indépendants
######## Notons que la taile de k-mer optimale est à déterminé (taille 1 : ça map partout et taille 100 (ie taille des reads : une toute petite erreurs empèche le mapping))

######### Un seuil est à déterminer sur la proportion de k-mer successif pour considérer le mapping du reads

In [14]:
100

100

In [15]:
def index_kmers(text,k):
    """
    construire dictionaire des indexes de kmers
    :param text: genome complet
    :param k: longeur de kmers
    :return: dictionaire des sous-textes et leurs indices de kmers
    #complexite = O(n)
    """
    index_dict = {}
    for i in range(len(text)-k+1):
        sous_texte = text[i:i+k]
        if sous_texte in index_dict:
            index_dict[sous_texte].append(i)
        else:
            index_dict[sous_texte] = [i]
    return index_dict

In [16]:
pattern = "abcdefghiabcn"
kmers = index_kmers(pattern,3)
print(f"kmers: {kmers}")

kmers: {'abc': [0, 9], 'bcd': [1], 'cde': [2], 'def': [3], 'efg': [4], 'fgh': [5], 'ghi': [6], 'hia': [7], 'iab': [8], 'bcn': [10]}


In [17]:
def suffix_list(T):
    """
    Compute the suffix list

    Args:
        T (str): string

    Return:
        list of strings: suffix list
    """
    suffix_list = [T[i:] for i in range(len(T))]
    sorted(suffix_list,reverse=True)
    return suffix_list

def suffix_table(T):
    """
    Compute the suffix tableolik

    Args:
        T (str): string

    Return:
        list of tuples (suffix,location): suffix table
    """
    suffix_list = [T[i:] for i in range(len(T))]
    suffix_table = sorted((e,i) for i,e in enumerate(suffix_list))
    suffix_array = []
    for suffix, index in suffix_table:
        suffix_array.append(index)
    return suffix_table, suffix_array

def BWT_suffix_table(T,end_of_string="$"):
    """
    Compute the BWT from the suffix table

    Args:
        T (str): string
        end_of_string (char): end of string character to append

    Return:
        bwt (str): BWT
    """
    T += end_of_string
    ST,SA= suffix_table(T)
    bwt = ""
    for s,i in ST:
        bwt += T[i-1]
    return(bwt)

In [18]:
def occurrence_indexer(S):
    """
    compute the occurrence index of a string
    :param S (str): 
    :return: list of occurrence indexes
    """
    K = []
    last_index = {}
    for s in S:
        if s not in last_index:
            last_index[s] = 0
        K.append(last_index[s])
        last_index[s] += 1
    return(K)

def last2first(counts,k,X):
    """
    find the index of the characters corrsponding to the occurrence index in the first column
    :param counts: 
    :param k: 
    :param X: 
    :return: 
    """
    return k + sum([counts[char] for char in counts if char < X])

def get_first_occurrence(L,X):
    """
    find the first occurence of a character in the bwt
    :param L: 
    :param X: 
    :return: 
    """
    for i,l in enumerate(L):
        if l == X:
            return(i)

def pattern_matching_BWT(SA, L ,pattern):
    """
    Search a pattern in a String using the BWT

    Args:
        SA (str): suffix array of the string
        L (str): BWT calculated by suffix array
        pattern (str): pattern

    Return:
        bool: true if the pattern is in the string
    """
    K = occurrence_indexer(L)
    counts = Counter(L)
    e = 0
    f = len(L)
    i = len(pattern) - 1
    while e < f and i >= 0:
        X = pattern[i]
        first_occurence_in_L_ef = get_first_occurrence(L[e:f],X)
        if first_occurence_in_L_ef is None:
            return False
        else:
            r = first_occurence_in_L_ef+e
        #print(first_occurence_in_L_ef,L[e:f],r,X,L[r])

        last_occurence_in_L_ef = get_first_occurrence(L[e:f][::-1],X) #find the first in reversed direction (the last)
        if last_occurence_in_L_ef is None:
            return False
        else:
            s = f-last_occurence_in_L_ef-1
        #print(last_occurence_in_L_ef,L[e:f],s,X,L[s])

        e = last2first(counts,K[r],X)
        f = last2first(counts,K[s],X)+1
        i -= 1
        # print(r,s,e,f)
    offsets =SA[e-1:f-1]
    offsets.sort()
    return offsets

In [19]:
pattern_matching_BWT('ACATACAGATG','CTA')

False

In [20]:
######Pour faire le bonnus ie avec les erreurs : on peut faire une boucle while en comptant les erreurs et en sortant dès qu'on atteint le seuil

In [21]:
def dc3(S,U):

  # Step 0

  #Creation T
  len_A = 4
  len_S = len(S)
  T = []
  P_0 = []
  P_1 = []
  P_2 = []
  P_12 = []
  for i in range (0, len_S):
    T.append(ord(S[i]))
  """print("T est", T)"""



  # Step 1 suite

  #Creation P_x
  for i in range (0, len_S):
    if i%3 == 0:
      P_0.append(i)
    elif i%3 == 1:
      P_1.append(i)
    elif i%3 == 2:
      P_2.append(i)
  T.append(0)
  T.append(0)
  T.append(0)
  if len_S%3 == 0:
    P_0.append(len_S)
  elif len_S%3 == 1:
    P_1.append(len_S)
  elif len_S%3 == 2:
    P_2.append(len_S)
  P_12 = P_1 + P_2
  """print("P_x est :", P_0, P_1, P_2, P_12)
  print("T est", T)"""

  #Creation R_12
  R_12 = []
  for i in range (0, len(P_12)):
    R_12.append((P_12[i], (T[P_12[i]], T[P_12[i]+1], T[P_12[i]+2])))
  """print("R_12 est: ", R_12)"""

  #Creation R_12_sorted + Index
  Inter = sorted(R_12, key=lambda x: x[1]) # note pour plustard : Implementer le Radix_sort, et pas utiliser la fonction sort (dif)
  R_12_sorted = [i[1] for i in Inter]
  Index = [i[0] for i in Inter]
  """print("R_12_sorted est: ", R_12_sorted)
  print("Index est: ", Index)"""

  #Creation Order
  Order = []
  current_rank = 1
  for i in range(len(R_12_sorted)):
      if i == 0:
          Order.append(current_rank)
      else:
          if R_12_sorted[i] == R_12_sorted[i - 1]:
              Order.append(current_rank)
          else:
              current_rank += 1
              Order.append(current_rank)

  """print("Order est: ", Order)"""


  #Calcule de T'
  def calculate_T_prime(R12, R12_sorted):
    T_prime = []
    rank_map = {}
    current_rank = 1
    for triplet in R12_sorted:
        if triplet not in rank_map:
            rank_map[triplet] = current_rank
            current_rank += 1
    for _, triplet in R12:
        T_prime.append(rank_map[triplet])
    return T_prime
  T_prime = calculate_T_prime(R_12, R_12_sorted)
  """print("ceci est R12", R_12)
  print("ceci est R12_sort", R_12_sorted)
  print("ceci est T'",T_prime)"""



dc3('abcabcacab',"coucou")


In [22]:
def dc3_bis(S):

  # Étape 0, Creation de T
  T = []
  for c in S:
      T.append(ord(c))
  T += [0, 0, 0]
  """
  print("T =", T)
  """

  # Étape 1 :

  # Creation de P0, P1, P2 et P1_2
  P0 = []
  P1 = []
  P2 = []
  for i in range(len(T)):
      if i % 3 == 1 and i + 2 < len(T):
          P1.append(i)
      elif i % 3 == 2 and i + 2 < len(T):
          P2.append(i)
      elif i % 3 == 0 and i + 2 < len(T):
          P0.append(i)
  P1_2 = P1 + P2
  """
  print("P0 =", P0)
  print("P1 =", P1)
  print("P2 =", P2)
  print("P1_2 =", P1_2)
  """

  # Creation de R1_2
  R1_2 = []
  for p in P1_2:
      triplet = (T[p], T[p+1], T[p+2])
      R1_2.append(triplet)
  """
  print("R1_2 =", R1_2)
  """

  # Creation de R1_2_sorted en utilisant un tri radix
  R1_2_with_p = list(zip(R1_2, P1_2))
  """
  print("R1_2_with_p =", R1_2_with_p)
  """
  R1_2_sorted = sorted(R1_2_with_p)
  """
  print("R1_2_sorted =", R1_2_sorted)
  """

  # Creation de Index1_2 et Order1_2
  rank = 0
  prev_triplet = None
  order_list = []
  p_to_rank = {}
  for triplet, p in R1_2_sorted:
      if triplet != prev_triplet:
          rank += 1
      p_to_rank[p] = rank
      prev_triplet = triplet
      order_list.append(rank)
  Index1_2 = [p for (triplet, p) in R1_2_sorted]
  Order1_2 = order_list
  print("Index1_2 =", Index1_2)
  print("Order1_2 =", Order1_2)

  # Création de T'
  T_prime = []
  for p in P1_2:
      T_prime.append(p_to_rank[p])
  print("T' =", T_prime)

  # Vérification des duplicatas dans Order1_2 pour décider de la récursivité
  if len(set(Order1_2)) < len(Order1_2):
      print("Des duplicatas ont été trouvés, récursion nécessaire.")
  else:
      print("Pas de duplicatas, pas de récursion nécessaire.")

  # Étape 2 : Calcul de R0
  R0 = []
  for p in P0:
      pair = (T[p], p_to_rank.get(p+1, 0))
      R0.append(pair)
  print("R0 =", R0)

  # Tri de R0
  R0_with_p = list(zip(R0, P0))
  R0_sorted = sorted(R0_with_p)
  print("R0 trié =", R0_sorted)
  Index0 = [p for (pair, p) in R0_sorted]
  print("Index0 =", Index0)

  # Étape 3 : Fusion de Index0 et Index1_2 pour obtenir le suffix array final
  SA = []
  i = j = 0
  while i < len(Index0) and j < len(Index1_2):
      idx0 = Index0[i]
      idx12 = Index1_2[j]
      if T[idx0:] < T[idx12:]:
          SA.append(idx0)
          i += 1
      else:
          SA.append(idx12)
          j += 1
  # Ajout des indices restants
  while i < len(Index0):
      SA.append(Index0[i])
      i += 1
  while j < len(Index1_2):
      SA.append(Index1_2[j])
      j += 1
  # Suppression des indices des sentinelles
  SA = [idx for idx in SA if idx < len(T) - 3]
  return ("Suffix Array final =", SA)

S = "abcabcacab"
print(dc3_bis(S))
print(dc3_bis("abracadabraabracadabra"))
print(dc3_bis("banana"))

Index1_2 = [10, 8, 1, 4, 2, 7, 5]
Order1_2 = [1, 2, 3, 3, 4, 4, 5]
T' = [3, 3, 4, 1, 4, 5, 2]
Des duplicatas ont été trouvés, récursion nécessaire.
R0 = [(97, 3), (97, 3), (97, 4), (98, 1)]
R0 trié = [((97, 3), 0), ((97, 3), 3), ((97, 4), 6), ((98, 1), 9)]
Index0 = [0, 3, 6, 9]
('Suffix Array final =', [8, 0, 3, 6, 9, 1, 4, 2, 7, 5])
Index1_2 = [22, 10, 7, 11, 14, 5, 16, 1, 8, 19, 4, 17, 20, 2, 13]
Order1_2 = [1, 2, 3, 3, 4, 5, 5, 6, 6, 6, 7, 8, 9, 10, 10]
T' = [6, 7, 3, 2, 10, 5, 6, 1, 10, 5, 6, 3, 4, 8, 9]
Des duplicatas ont été trouvés, récursion nécessaire.
R0 = [(97, 6), (97, 7), (100, 3), (114, 2), (98, 10), (99, 5), (97, 6), (97, 1)]
R0 trié = [((97, 1), 21), ((97, 6), 0), ((97, 6), 18), ((97, 7), 3), ((98, 10), 12), ((99, 5), 15), ((100, 3), 6), ((114, 2), 9)]
Index0 = [21, 0, 18, 3, 12, 15, 6, 9]
('Suffix Array final =', [21, 10, 7, 11, 0, 18, 14, 3, 5, 16, 12, 1, 8, 19, 15, 4, 17, 6, 20, 9, 2, 13])
Index1_2 = [5, 1, 4, 2]
Order1_2 = [1, 2, 3, 4]
T' = [2, 3, 4, 1]
Pas de dupli

In [23]:
def dc3_quasi_final(S):
    print('S',S)
    # Étape 0, Creation de T
    T = []
    if len(S) == 0:
        return []
    if isinstance(S[0], str):  # On vérifie que S contient des chaînes
        for c in S:
            T.append(ord(c))
    else:
        T = S.copy()  # Si c'est déjà une liste d'entiers, on la copie
    T += [0, 0, 0]  # Ajouter les sentinelles
    
    # Étape 1 :

    # Creation de P0, P1, P2 et P1_2
    P0, P1, P2 = [], [], []
    for i in range(len(T)-2):
        if i % 3 == 1:
            P1.append(i)
        elif i % 3 == 2:
            P2.append(i)
        elif i % 3 == 0 and i<len(T)-3:
            P0.append(i)
    P1_2 = P1 + P2

    # Creation de R1_2
    R1_2 = []
    for p in P1_2:
        triplet = (T[p], T[p+1], T[p+2])
        R1_2.append(triplet)

    # Tri de R1_2 avec les positions
    R1_2_with_p = list(zip(R1_2, P1_2))
    R1_2_sorted = sorted(R1_2_with_p)
    #print("R1_2_sorted",R1_2_sorted)
    # Creation de Index1_2 et Order1_2
    rank = 0
    prev_triplet = None
    p_to_rank = {}
    order_list = []
    for triplet, p in R1_2_sorted:
        if triplet != prev_triplet:
            rank += 1
        p_to_rank[p] = rank
        prev_triplet = triplet
        order_list.append(rank)
    Index1_2 = [p for (triplet, p) in R1_2_sorted]
    Order1_2 = order_list

    # Création de T'
    T_prime = []
    for p in P1_2:
        T_prime.append(p_to_rank[p])

    # Vérification des duplicatas dans Order1_2 pour décider de la récursivité
    if len(set(Order1_2)) < len(Order1_2):  # Il y a des duplicatas
        #print('rec')
        Index_prime0_1_2 = dc3_quasi_final(T_prime)  # Récursion
        Index1_2 = [P1_2[i] for i in Index_prime0_1_2]
    
    # Étape 2 : Calcul de R0
    R0 = []
    for p in P0:
        pair = (T[p], p_to_rank.get(p+1))
        R0.append(pair)
        
    # Tri de R0
    R0_with_p = list(zip(R0, P0))
    R0_sorted = sorted(R0_with_p)
    Index0 = [p for (pair, p) in R0_sorted]

    # Étape 3 : Fusion de Index0 et Index1_2 pour obtenir le suffix array final
    
    def firstin(l,a,b):#######################################################################j'espère que je ne suis pas en train d'augmenter la complexité !
        for i in range(len(l)):
            if l[i][1]==a:
                return (1,0),a
            if l[i][1]==b:
                return (0,1),b
        print("ERREUR",a,b,l)
        return
    
    SA = []
    i = 0
    j = 0
    while i < len(Index0) and j < len(Index1_2):
        a = Index0[i]
        b = Index1_2[j]
        if T[a] < T[b] :
            SA.append(a)
            i += 1
        elif T[a] > T[b]:
            SA.append(b)
            j += 1
        else :
            if b%3==1:
                i_ou_j,a_ou_b=firstin(R1_2_sorted,a+1,b+1)
                SA.append(a_ou_b-1)
                i+=i_ou_j[0]
                j+=i_ou_j[1]
            elif b%3==2:
                if T[a+1] < T[b+1]:
                    SA.append(a)
                    i += 1
                elif T[a+1] > T[b+1]:
                    SA.append(b)
                    j += 1
                else:
                    i_ou_j,a_ou_b=firstin(R1_2_sorted,a+2,b+2)
                    SA.append(a_ou_b-2)
                    i+=i_ou_j[0]
                    j+=i_ou_j[1]
        print(a,b,SA)

    # Ajout des indices restants
    while i < len(Index0):
        SA.append(Index0[i])
        i += 1
        print("_0",SA)
    while j < len(Index1_2):
        SA.append(Index1_2[j])
        j += 1
        print("1_2",SA)
    # Suppression des indices des sentinelles
    SA = [idx for idx in SA if idx < len(T) - 3]
    return SA

test="aaaaaaaaaaaaaaaaaaa"
SA=dc3_quasi_final(test)
print(SA,len(SA))
naive_suffix_array(test)

S aaaaaaaaaaaaaaaaaaa
S [3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2]
S [6, 4, 6, 5, 1, 6, 2, 6, 3]
6 4 [4]
6 8 [4, 6]
3 8 [4, 6, 8]
3 1 [4, 6, 8, 1]
3 5 [4, 6, 8, 1, 3]
0 5 [4, 6, 8, 1, 3, 5]
0 7 [4, 6, 8, 1, 3, 5, 7]
0 2 [4, 6, 8, 1, 3, 5, 7, 0]
1_2 [4, 6, 8, 1, 3, 5, 7, 0, 2]
6 13 [13]
6 5 [13, 6]
12 5 [13, 6, 12]
3 5 [13, 6, 12, 5]
3 11 [13, 6, 12, 5, 11]
3 4 [13, 6, 12, 5, 11, 4]
3 10 [13, 6, 12, 5, 11, 4, 10]
3 2 [13, 6, 12, 5, 11, 4, 10, 3]
9 2 [13, 6, 12, 5, 11, 4, 10, 3, 9]
0 2 [13, 6, 12, 5, 11, 4, 10, 3, 9, 2]
0 8 [13, 6, 12, 5, 11, 4, 10, 3, 9, 2, 8]
0 1 [13, 6, 12, 5, 11, 4, 10, 3, 9, 2, 8, 0]
1_2 [13, 6, 12, 5, 11, 4, 10, 3, 9, 2, 8, 0, 1]
1_2 [13, 6, 12, 5, 11, 4, 10, 3, 9, 2, 8, 0, 1, 7]
18 19 [19]
18 17 [19, 18]
0 17 [19, 18, 17]
0 16 [19, 18, 17, 16]
0 14 [19, 18, 17, 16, 0]
3 14 [19, 18, 17, 16, 0, 3]
6 14 [19, 18, 17, 16, 0, 3, 6]
9 14 [19, 18, 17, 16, 0, 3, 6, 9]
12 14 [19, 18, 17, 16, 0, 3, 6, 9, 12]
15 14 [19, 18, 17, 16, 0, 3, 6, 9, 12, 15]
1_2 [19, 18, 17, 16, 0, 3, 

NameError: name 'naive_suffix_array' is not defined

In [29]:
###################Verification de DC3

def naive_suffix_array(T):
    suffixes = [(T[i:], i) for i in range(len(T))]
    suffixes.sort()
    return [suffix[1] for suffix in suffixes]

def compare_suffix_arrays(dc3_result, naive_result):
    if dc3_result == naive_result:
        print("Succès : Les suffix arrays sont identiques !")
        print("Résultat DC3 :", dc3_result)
        print("Résultat naïf :", naive_result)
    else:
        print("Erreur : Les suffix arrays sont différents.")
        print("Résultat DC3 :", dc3_result)
        print("Résultat naïf :", naive_result)

def test_dc3(dc3_bis_bis):
    test_strings = [
        "abracadabraabracadabraaaaaaafytdxqubgfshfdddddddddddd",
        "abaabaaba",
        "banana",
        "abracadabra",
        "abcabcabc", 
        "aaaaaaaaaa",
        "mississippi",
        "",
        "a",
    ]

    for i, T in enumerate(test_strings):
        print(f"Test {i+1}: Chaîne '{T}'")
        dc3_result = dc3_bis_bis(T)
        naive_result = naive_suffix_array(T)
        compare_suffix_arrays(dc3_result, naive_result)
        print()

#test_dc3(dc3_quasi_final)

In [25]:
def DC3_internet(x : list[int], asize : int) -> list[int]:
    if not x :
        return []
    if type(x[0]) == str:
        x = [ord(y) for y in x]
        asize = 256
    SA12 = [i for i in range(len(x)) if i % 3 != 0]
    
    SA12 = radix3(x, asize, SA12)
    new_alpha = collect_alphabet(x, SA12)
    if len(new_alpha) < len(SA12):
        # Recursively sort SA12.
        # Construct the u string and compute its suffix array
        u = build_u(x, new_alpha)
        # For the recursion, remember that the real alphabet has
        # two sentinels, so + 2
        sa_u = DC3_internet(u, len(new_alpha) + 2)
        # Then map u's suffix array back to a sorted SA12
        m = len(sa_u) // 2
        SA12 = [u_idx(i, m) for i in sa_u if i != m]

    # Special case if the last index is class 0. Then the
    # following class 1 isn't there, but we should treat it
    # as the smallest string in the class.
    SA3 = ([len(x) - 1] if len(x) % 3 == 1 else []) + \
          [i - 1 for i in SA12 if i % 3 == 1]
    SA3 = bucket_sort(x, asize, SA3)
    return merge(x, SA12, SA3)

def safe_idx(x : list[int], i : int) -> int:
    "Hack to get zero if we index beyond the end."
    return 0 if i >= len(x) else x[i]

def symbcount(x : list[int], asize : int) -> list[int]:
    "Count how often we see each character in the alphabet."
    # This is what collections.Counter does, but we need the
    # alphabet to be sorted integers, so we do it manually.
    counts = [0] * asize
    for c in x:
        counts[c] += 1
    return counts

def cumsum(counts : list[int]) -> list[int]:
    "Compute the cumulative sum from the character count."
    res, acc = [0] * len(counts), 0
    for i, k in enumerate(counts):
        res[i] = acc
        acc += k
    return res

def bucket_sort(x : list[int], asize : int,
                idx : list[int], offset : int = 0) -> list[int]:
    "Sort indices in idx according to x[i + offset]."
    sort_symbs = [safe_idx(x, i + offset) for i in idx]
    counts = symbcount(sort_symbs, asize)
    buckets = cumsum(counts)
    out = [None] * len(idx)
    for i in idx:
        bucket = safe_idx(x, i + offset)
        out[buckets[bucket]] = i
        buckets[bucket] += 1
    return out

def radix3(x : list[int], asize : int, idx : list[int]) -> list[int]:
    "Sort indices in idx according to their first three letters in x."
    idx = bucket_sort(x, asize, idx, 2)
    idx = bucket_sort(x, asize, idx, 1)
    return bucket_sort(x, asize, idx)

TRIPLET = tuple[int,int,int]
TRIPLET_DICT = dict[TRIPLET,int]

def triplet(x : list[int], i : int) -> TRIPLET:
    "Extract the triplet (x[i],x[i+1],x[i+2])."
    return (safe_idx(x, i), safe_idx(x, i + 1), safe_idx(x, i + 2))

def collect_alphabet(x : list[int], idx : list[int]) -> TRIPLET_DICT:
    "Map the triplets starting at idx to a new alphabet."
    # I use 0 for the terminal sentinel and 1 for the 
    # separator, so I start the alphabet at 2, thus the + 2 later.
    # I'm using a dictionary for the alphabet, but you can build
    # it more efficiently by looking at the previous triplet in the
    # sorted SA12. It won't affect the asymptotic running time,
    # though.
    alpha = {}
    for i in idx:
        trip = triplet(x, i)
        if trip not in alpha:
            alpha[trip] = len(alpha) + 2 # +2 to reserve sentinels
    return alpha

def build_u(x : list[int], alpha : TRIPLET_DICT) -> list[int]:
    "Construct u string, using 1 as central sentinel."
    # By putting the i % 3 == 1 indices first, we know that the central
    # sentinel will always be at len(u) // 2.
    return [ *(alpha[triplet(x, i)] for i in range(1, len(x), 3)),
           1,
             *(alpha[triplet(x, i)] for i in range(2, len(x), 3)) ]

def u_idx(i : int, m : int) -> int:
    "Map indices in u back to indices in the original string."
    if i < m: return 1 + 3 * i
    else: return 2 + 3 * (i - m - 1)

def merge(x : list[int], SA12 : list[int], SA3 : list[int]) -> list[int]:
    "Merge the suffixes in sorted SA12 and SA3."
    # I'm using a dict here, but you can use a list with a little
    # arithmetic
    ISA = { SA12[i]: i for i in range(len(SA12)) }
    SA = []
    i, j = 0, 0
    while i < len(SA12) and j < len(SA3):
        if less(x, SA12[i], SA3[j], ISA):
            SA.append(SA12[i])
            i += 1
        else:
            SA.append(SA3[j])
            j += 1
    SA.extend(SA12[i:])
    SA.extend(SA3[j:])
    return SA

def less(x : list[int], i : int, j : int, ISA : dict[int,int]) -> bool:
    "Check if x[i:] < x[j:] using the inverse suffix array for SA12."
    a, b = safe_idx(x, i), safe_idx(x, j)
    if a < b: return True
    if a > b: return False
    if i % 3 != 0 and j % 3 != 0: return ISA[i] < ISA[j]
    return less(x, i + 1, j + 1, ISA)

In [26]:
SA=DC3_internet(genome,1000)

In [27]:
print(SA[:100])

[174571, 178118, 397728, 613032, 346794, 313120, 170534, 243148, 242835, 307343, 513411, 343771, 537995, 192442, 103729, 174572, 178119, 121149, 166895, 397729, 90604, 144293, 613033, 141886, 242198, 244339, 598852, 346795, 527533, 313121, 170535, 208255, 437753, 194455, 71810, 515716, 470523, 535107, 255424, 527687, 437035, 243149, 242836, 307344, 409697, 118615, 104965, 444426, 309001, 437007, 361143, 178718, 617296, 375523, 143490, 246544, 93097, 95218, 312502, 588175, 27134, 401025, 98885, 513412, 343772, 531407, 230657, 537996, 178451, 167810, 277171, 570929, 330833, 192443, 382947, 103730, 464972, 6466, 5777, 50917, 331169, 132159, 276764, 513285, 92559, 164519, 529218, 283589, 188591, 174573, 178120, 121150, 78483, 51458, 166896, 397730, 90605, 144294, 592483, 461896]


In [28]:
def query_kmers(pattern,T,k):
    kmers_dict = index_kmers(pattern,k)
    res = {}
    for kmer,index in kmers_dict.items():
        offset = pattern_matching_BWT(T,kmer)
        if offset:
            print(offset)
            res[kmer] = offset
        else:
            res[kmer] = None
    return res

In [29]:
g enome = "TGAACCCTaaaacctaaaccctaaaccctaaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccctaaaccctaaaccctgaaccctaaaccctgaaacctaaaccctgaaccctaaaccctgaaccctgaaccctaaccctaaaccctaaacctaaaaccctgaaccctaaaccctgaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccctgaaccctaaaacctaaaccctaaaccctaaaccctaaaccctgaacctaAACCTAAAACCTAAAACCtaaaaccctgaacccttacttttcatttcttcttcttatcttcttacttttcattcttta"
pattern = "AACCTAAAACCTAAAACC"
query_kmers(pattern,genome,5)

[322, 329]
[323, 330]
[324, 331]
[325, 332]
[326, 333]
[327, 334]
[328, 335]


{'AACCT': [322, 329],
 'ACCTA': [323, 330],
 'CCTAA': [324, 331],
 'CTAAA': [325, 332],
 'TAAAA': [326, 333],
 'AAAAC': [327, 334],
 'AAACC': [328, 335]}

In [30]:
# Importer le fichier BAM
def read_fichier_BAM():
  bam_dict = {}
  bam_file = pysam.AlignmentFile(bam_path, "rb")
    for read in bam_file:
        read_name= read.query_name
        read_seq = read.query_sequence
        read_position = read.reference_start
        read_quality = read.mapping_quality
        read_is_reverse = read.is_reverse
        bam_info[read_name] = {"sequence": read_seq,
                               "position": read_position,
                               "quality": read_quality,
                               "reversed_strand":read_is_reverse}
    bam_file.close()
    return bam_dict

IndentationError: unexpected indent (2947314226.py, line 5)

In [ ]:
def main(genome,reads,erreur_max = 2,output_file = "./tmp/reads.txt"):
    """
    realize the read for each sequence to each chromosome of the genome
    :param genome (list): lists of chromosomes
    :param reads (list): list of reads
    :erreur_max (int): number maximum of error
    :output_file (string): output path
    :return: liste des listes de positions exactes trouvées
    """
    file = open(output_file, "w")
    results= []
    for i_chromosome,chromosome in enumerate(genome):
        
        SA = DC3(chromosome,256)
        L = BWT_suffix_table(chromosome)
        ##### il faut ajouter aussi la fonction pour inverser le reads
        print(f"Dans la chromosome {i_chromosome+1} :")

        for read in reads:
            reverse_read = reverse_complement(str(read))
            print("\t straight strand")
            fit_straight=analyse_results(chromosome,str(read),query_kmers(str(read),SA,L,5),erreur_max) 
            if len(fit_straight)>0:
                file.write(str(read)+","+str(i_chromosome)+","+",".join(map(str,fit_straight))+","+"0"+"\n")
            results.append(fit_straight)
            print("\t reversed strand")
            fit_reversed=analyse_results(chromosome,reverse_read,query_kmers(reverse_read,SA,L,5),erreur_max)   
            if len(fit_reversed)>0:
                file.write(reverse_read+","+str(i_chromosome)+","+",".join(map(str,fit_reversed))+","+"0"+"\n")
            results.append(fit_reversed)
        print("_________________")
    file.close()
    print("#########finished searching##########")
    return results

In [ ]:
def analyse_results(genome, pattern,kmers_res,max_erreurs=2):
    """
    analyse the results of the mapping of kmers to a genome
    :param genome (str): genome from a chromosome 
    :param pattern (str): pattern/read 
    :param kmers_res: result of query_kmers
    :param max_erreurs (int): (default 2) maximum number of erreurs 
    :return: list of positions that found the pattern (complete) in genome
    """
    kmers,all_positions = kmers_res
    fit = []
    variations = []
    k = len(kmers[0])
    for current_kmer,position in enumerate(all_positions):
        for pos in position:
            aligment = [pos]
            for next_kmer, next_position in enumerate(all_positions,start=current_kmer):
                expected_pos = pos + next_kmer - current_kmer
                if expected_pos in next_position:
                    if expected_pos not in aligment:
                        aligment.append(expected_pos)
            if len(aligment) == len(all_positions):
                fit.append(aligment[0])
            else: 
                if len(aligment) > 1:
                    start_i_genome = aligment[0] - current_kmer
                    end_i_genome = start_i_genome + len(pattern)
                    
                    if start_i_genome >= 0 and end_i_genome <= len(genome):
                        genome_subseq = genome[start_i_genome:end_i_genome]
                        erreurs = hamming_distance(genome_subseq,pattern)
                        if erreurs <= max_erreurs:
                            if start_i_genome not in variations:
                                variations.append(start_i_genome)
    variations = list(set(fit)-set(variations))
    if len(fit)>=1:
        print(f"La séquence trouvée dans le génome avec {len(fit)} occurences")
        print(f"Les positions trouvées sont : {fit}")
    else:
        print("aucune séquence exacte trouvée dans le génome")
    if len(variations)>=1:
        print(f"trouvées les séquences variés dont {max_erreurs} d'erreurs max avec {len(variations)} occurences")
        print(f"Les positions trouvées sont : {variations}")

    return fit

In [ ]:
def hamming_distance(seq1, seq2):
    """
    calculate the hamming distance between two strings
    :param seq1 (str):
    :param seq2 (str): 
    :return: 
    """
    return sum(1 for a, b in zip(seq1, seq2) if a != b)